<a href="https://colab.research.google.com/github/tamtran28/cautruc_tieuchitindung/blob/main/bangoc_CIF_chu_tindung_dot3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

# 1. Đọc tất cả file HDV chi tiết CKH (*.xlsx)
crm4_files = glob.glob("/content/vb/CRM4_Du_no_theo_tai_san_dam_bao_ALL*.xls")  # /content/vb/CRM4_Du_no_theo_tai_san_dam_bao_ALL0.xlshoặc *.xls nếu file cũ/content/vb

crm32_files = glob.glob("/content/vb/RPT_CRM_32*.xls")
df_muc_dich_file = pd.read_excel("/content/vb/CODE_MDSDV4.xlsx")
df_code_tsbd_file = pd.read_excel("/content/vb/CODE_LOAI TSBD.xlsx")

# Các cột cần lấy từ từng file
# Đọc file
# 1.1. Ghép tất cả file
df_crm4_ghep = [pd.read_excel(f) for f in crm4_files]
df_crm4 = pd.concat(df_crm4_ghep, ignore_index=True)

# 1.2. Ghép tất cả file CRM32
df_crm32_ghep = [pd.read_excel(f) for f in crm32_files]
df_crm32 = pd.concat(df_crm32_ghep, ignore_index=True)

In [ ]:
for df in [df_crm4]:
    if 'CIF_KH_VAY' in df.columns:
        df['CIF_KH_VAY'] = pd.to_numeric(df['CIF_KH_VAY'], errors='coerce')  # về dạng float
        df['CIF_KH_VAY'] = df['CIF_KH_VAY'].dropna().astype('int64').astype(str)  # về int rồi về str

In [ ]:
for df in [df_crm32]:
    if 'CUSTSEQLN' in df.columns:
        df['CUSTSEQLN'] = pd.to_numeric(df['CUSTSEQLN'], errors='coerce')  # về dạng float
        df['CUSTSEQLN'] = df['CUSTSEQLN'].dropna().astype('int64').astype(str)  # về int rồi về str

In [ ]:
df_crm4_list =df_crm4_ghep.copy()
df_crm32_list=df_crm32_ghep.copy()
df_muc_dich = df_muc_dich_file.copy()
df_code_tsbd = df_code_tsbd_file.copy()

In [ ]:
chi_nhanh = input("Nhập tên chi nhánh hoặc mã SOL cần lọc (ví dụ: HANOI hoặc 001): ").strip().upper()

# ✅ Lọc dữ liệu theo BRCD chứa chuỗi nhập vào
df_crm4_filtered = df_crm4[df_crm4['BRANCH_VAY'].astype(str).str.upper().str.contains(chi_nhanh)]
df_crm32_filtered = df_crm32[df_crm32['BRCD'].astype(str).str.upper().str.contains(chi_nhanh)]

# In preview
print(f"📌 Số dòng sau khi lọc FTP theo chi nhánh '{chi_nhanh}': {len(df_crm4_filtered)}")
# print(df_crm4_filtered.head())


Nhập tên chi nhánh hoặc mã SOL cần lọc (ví dụ: HANOI hoặc 001): 1610
📌 Số dòng sau khi lọc FTP theo chi nhánh '1610': 327


In [ ]:
# Giả sử cột chứa code là 'CODE CAP 2' và mã phân loại là 'CODE'
df_code_tsbd = df_code_tsbd[['CODE CAP 2', 'CODE']]
df_code_tsbd.columns = ['CAP_2', 'LOAI_TS']

In [ ]:
# 1. Tạo bản sao nhỏ gọn từ df_code_tsbd chỉ lấy 2 cột cần dùng
df_tsbd_code = df_code_tsbd[['CAP_2', 'LOAI_TS']].drop_duplicates()

# 2. Merge để lấy LOAI_TS vào df_crm4_filtered
df_crm4_filtered = df_crm4_filtered.merge(df_tsbd_code, how='left', on='CAP_2')

# 3. Gán 'Không TS' nếu CAP_2 rỗng hoặc NaN
df_crm4_filtered['LOAI_TS'] = df_crm4_filtered.apply(
    lambda row: 'Không TS' if pd.isna(row['CAP_2']) or str(row['CAP_2']).strip() == '' else row['LOAI_TS'],
    axis=1
)

# 4. Ghi chú 'MỚI' nếu CAP_2 có nhưng LOAI_TS vẫn NaN (không khớp trong bảng mã)
df_crm4_filtered['GHI_CHU_TSBD'] = df_crm4_filtered.apply(
    lambda row: 'MỚI' if str(row['CAP_2']).strip() != '' and pd.isna(row['LOAI_TS']) else '',
    axis=1
)


In [ ]:
df_vay_4 = df_crm4_filtered.copy()

In [ ]:
# Bỏ các dòng có LOAI là "Bao lanh" hoặc "LC"
df_vay = df_vay_4[~df_vay_4['LOAI'].isin(['Bao lanh', 'LC'])]

In [ ]:
pivot_ts = df_vay.pivot_table(
    index='CIF_KH_VAY',
    columns='LOAI_TS',
    values='TS_KW_VND',
    aggfunc='sum',
    fill_value=0
).add_suffix(' (Giá trị TS)').reset_index()

pivot_no = df_vay.pivot_table(
    index='CIF_KH_VAY',
    columns='LOAI_TS',
    values='DU_NO_PHAN_BO_QUY_DOI',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [ ]:
pivot_merge = pivot_no.merge(pivot_ts, on='CIF_KH_VAY', how='left')
pivot_merge['GIÁ TRỊ TS'] = pivot_ts.drop(columns='CIF_KH_VAY').sum(axis=1)
pivot_merge['DƯ NỢ'] = pivot_no.drop(columns='CIF_KH_VAY').sum(axis=1)

In [ ]:
df_info = df_crm4_filtered[['CIF_KH_VAY', 'TEN_KH_VAY', 'CUSTTPCD', 'NHOM_NO']].drop_duplicates(subset='CIF_KH_VAY')
pivot_final = df_info.merge(pivot_merge, on='CIF_KH_VAY', how='left')
pivot_final = pivot_final.reset_index().rename(columns={'index': 'STT'})
pivot_final['STT'] += 1

In [ ]:
cols_order = ['STT', 'CUSTTPCD', 'CIF_KH_VAY', 'TEN_KH_VAY', 'NHOM_NO'] + \
             sorted([col for col in pivot_merge.columns if col not in ['CIF_KH_VAY', 'GIÁ TRỊ TS', 'DƯ NỢ'] and '(Giá trị TS)' not in col]) + \
             sorted([col for col in pivot_merge.columns if '(Giá trị TS)' in col]) + \
             ['DƯ NỢ', 'GIÁ TRỊ TS']

pivot_final = pivot_final[cols_order]

In [ ]:
print(pivot_final)

     STT CUSTTPCD CIF_KH_VAY           TEN_KH_VAY  NHOM_NO          BĐS  GTCG  \
0      1  Ca nhan  103992203            TRAN TUAN        5   21075840.0   0.0   
1      2  Ca nhan  116606267   LUONG THI NGOC ANH        1  144000000.0   0.0   
2      3  Ca nhan  112936963      NGUYEN HOANG HA        1  239050000.0   0.0   
3      4  Ca nhan  106071335     LE THI TUYET NGA        1  107620000.0   0.0   
4      5  Ca nhan  118513828   TRAN THI THU HUYEN        1  148897000.0   0.0   
..   ...      ...        ...                  ...      ...          ...   ...   
181  182  Ca nhan  118752694        DANG THANH VU        1          0.0   0.0   
182  183  Ca nhan  121254980    LE THI NGOC TRANG        1          0.0   0.0   
183  184  Ca nhan  121278521        LE THI MY HOI        1          0.0   0.0   
184  185  Ca nhan  106784138     PHAM THI ANH THU        1          0.0   0.0   
185  186  Ca nhan  110644865  PHAM THI THANH LOAN        1          0.0   0.0   

       Không TS  MMTB  PTVT

In [ ]:
# 1. Chuẩn hóa mã CAP_PHE_DUYET trong df_crm4_filtered
df_crm32_filtered = df_crm32_filtered.copy()
df_crm32_filtered['MA_PHE_DUYET'] = df_crm32_filtered['CAP_PHE_DUYET'].astype(str).str.split('-').str[0].str.strip().str.zfill(2)


In [ ]:
# 2. Tạo danh sách CIF có cấp phê duyệt cấp C
ma_cap_c = [f"{i:02d}" for i in range(1, 8)] + [f"{i:02d}" for i in range(28, 32)]
list_cif_cap_c = df_crm32_filtered[df_crm32_filtered['MA_PHE_DUYET'].isin(ma_cap_c)]['CUSTSEQLN'].unique()

In [ ]:
list_co_cau = ['ACOV1', 'ACOV3', 'ATT01', 'ATT02', 'ATT03', 'ATT04',
               'BCOV1', 'BCOV2', 'BTT01', 'BTT02', 'BTT03',
               'CCOV2', 'CCOV3', 'CTT03', 'RCOV3', 'RTT03']
cif_co_cau = df_crm32_filtered[
    df_crm32_filtered['SCHEME_CODE'].isin(list_co_cau)
]['CUSTSEQLN'].unique()

ghi chu

In [ ]:
# Giả sử cột chứa code là 'CODE CAP 2' và mã phân loại là 'CODE'
df_muc_dich_vay = df_muc_dich[['CODE_MDSDV4', 'GROUP']]
df_muc_dich_vay.columns = ['MUC_DICH_VAY_CAP_4', 'MUC DICH']


In [ ]:
# 1. Tạo bản sao nhỏ gọn từ df_code_tsbd chỉ lấy 2 cột cần dùng
df_muc_dich = df_muc_dich_vay[['MUC_DICH_VAY_CAP_4', 'MUC DICH']].drop_duplicates()

# 2. Merge để lấy LOAI_TS vào df_crm4_filtered
df_crm32_filtered = df_crm32_filtered.merge(df_muc_dich_vay, how='left', on='MUC_DICH_VAY_CAP_4')
df_crm32_filtered['MUC DICH'] = df_crm32_filtered['MUC DICH'].fillna('(blank)')
# 4. Ghi chú 'MỚI' nếu CAP_2 có nhưng LOAI_TS vẫn NaN (không khớp trong bảng mã)
df_crm32_filtered['GHI_CHU_TSBD'] = df_crm32_filtered.apply(
    lambda row: 'MỚI' if str(row['MUC_DICH_VAY_CAP_4']).strip() != '' and pd.isna(row['MUC DICH']) else '',
    axis=1
)


In [ ]:
df_crm32_filtered.to_excel('df_crm32_filtered.xlsx', index=False)

cr32

In [ ]:
pivot_mucdich = df_crm32_filtered.pivot_table(
    index='CUSTSEQLN',
    columns='MUC DICH',
    values='DU_NO_QUY_DOI',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [ ]:
pivot_mucdich['DƯ NỢ CRM32'] = pivot_mucdich.drop(columns='CUSTSEQLN').sum(axis=1)

In [ ]:
# 1. Đổi tên CUSTSEQLN để đồng bộ khóa ghép
pivot_final_CRM32 = pivot_mucdich.rename(columns={'CUSTSEQLN': 'CIF_KH_VAY'})
# 2. Ghép bảng chính với CRM32
pivot_full = pivot_final.merge(pivot_final_CRM32, on='CIF_KH_VAY', how='left')
pivot_full.fillna(0, inplace=True)

In [ ]:
# 1. Tính lệch dư nợ
pivot_full['LECH'] = pivot_full['DƯ NỢ'] - pivot_full['DƯ NỢ CRM32']
pivot_full['LECH'] = pivot_full['LECH'].fillna(0)

# 2. Lọc các CIF bị lệch
cif_lech = pivot_full[pivot_full['LECH'] != 0]['CIF_KH_VAY'].unique()

# 3. Lọc CRM4: loại bỏ 'Cho vay'
df_crm4_blank = df_crm4_filtered[~df_crm4_filtered['LOAI'].isin(['Cho vay', 'Bao lanh', 'LC'])].copy()


# 4. Tổng dư nợ theo CIF bị lệch
du_no_bosung = (
    df_crm4_blank[df_crm4_blank['CIF_KH_VAY'].isin(cif_lech)]
    .groupby('CIF_KH_VAY', as_index=False)['DU_NO_PHAN_BO_QUY_DOI']
    .sum()
    .rename(columns={'DU_NO_PHAN_BO_QUY_DOI': '(blank)'})
)

# 5. Gộp dư nợ bổ sung vào pivot_full
pivot_full = pivot_full.merge(du_no_bosung, on='CIF_KH_VAY', how='left')
pivot_full['(blank)'] = pivot_full['(blank)'].fillna(0)

# 6. Cộng vào DƯ NỢ CRM32
pivot_full['DƯ NỢ CRM32'] = pivot_full['DƯ NỢ CRM32'] + pivot_full['(blank)']

# 7. Sắp xếp lại cột: đưa (blank) lên trước DƯ NỢ CRM32
cols = list(pivot_full.columns)
if '(blank)' in cols and 'DƯ NỢ CRM32' in cols:
    cols.insert(cols.index('DƯ NỢ CRM32'), cols.pop(cols.index('(blank)')))
    pivot_full = pivot_full[cols]

# 8. Cập nhật lại LECH sau khi cộng bổ sung
# pivot_full['LECH'] = pivot_full['DƯ NỢ'] - pivot_full['DƯ NỢ CRM32']


In [ ]:
pivot_full['Nợ nhóm 2'] = pivot_full['NHOM_NO'].apply(lambda x: 'x' if str(x).strip() == '2' else '')
pivot_full['Nợ xấu'] = pivot_full['NHOM_NO'].apply(lambda x: 'x' if str(x).strip() in ['3', '4', '5'] else '')


In [ ]:
pivot_full['Chuyên gia PD cấp C duyệt'] = pivot_full['CIF_KH_VAY'].apply(lambda x: 'x' if x in list_cif_cap_c else '')

In [ ]:
pivot_full['NỢ CƠ_CẤU'] = pivot_full['CIF_KH_VAY'].apply(
    lambda x: 'x' if x in cif_co_cau else ''
)


In [ ]:
# Lọc dữ liệu từ CRM4 (LOAI = Bao lanh hoặc LC)
#df_crm4_filtered['LOAI'] = df_crm4_filtered['LOAI'].str.strip().str.upper()
df_baolanh = df_crm4_filtered[df_crm4_filtered['LOAI'] == 'Bao lanh']
df_lc = df_crm4_filtered[df_crm4_filtered['LOAI'] == 'LC']

# Gộp tổng dư nợ theo CIF (tạo DataFrame, KHÔNG dùng dict)
df_baolanh_sum = df_baolanh.groupby('CIF_KH_VAY', as_index=False)['DU_NO_PHAN_BO_QUY_DOI'].sum()
df_baolanh_sum = df_baolanh_sum.rename(columns={'DU_NO_PHAN_BO_QUY_DOI': 'DƯ_NỢ_BẢO_LÃNH'})

df_lc_sum = df_lc.groupby('CIF_KH_VAY', as_index=False)['DU_NO_PHAN_BO_QUY_DOI'].sum()
df_lc_sum = df_lc_sum.rename(columns={'DU_NO_PHAN_BO_QUY_DOI': 'DƯ_NỢ_LC'})

# Trước khi merge, nếu pivot_full đã có cột DƯ_NỢ_BẢO_LÃNH thì xóa đi để tránh trùng
if 'DƯ_NỢ_BẢO_LÃNH' in pivot_full.columns:
    pivot_full = pivot_full.drop(columns=['DƯ_NỢ_BẢO_LÃNH'])

# Merge dư nợ bảo lãnh
pivot_full = pivot_full.merge(df_baolanh_sum, on='CIF_KH_VAY', how='left')

# Merge dư nợ LC (tương tự, tránh trùng tên)
if 'DƯ_NỢ_LC' in pivot_full.columns:
    pivot_full = pivot_full.drop(columns=['DƯ_NỢ_LC'])

pivot_full = pivot_full.merge(df_lc_sum, on='CIF_KH_VAY', how='left')

# Fill NaN nếu không có dư nợ
pivot_full['DƯ_NỢ_BẢO_LÃNH'] = pivot_full['DƯ_NỢ_BẢO_LÃNH'].fillna(0)
pivot_full['DƯ_NỢ_LC'] = pivot_full['DƯ_NỢ_LC'].fillna(0)


giai ngan 1 ty

In [ ]:
df_giai_ngan = pd.read_excel('/content/vb/Giai_ngan_tien_mat_1_ty.xls')

In [ ]:
import pandas as pd

# 1. Chuẩn hóa dữ liệu: ép kiểu chuỗi, loại bỏ khoảng trắng
df_crm32_filtered['KHE_UOC'] = df_crm32_filtered['KHE_UOC'].astype(str).str.strip()
df_crm32_filtered['CUSTSEQLN'] = df_crm32_filtered['CUSTSEQLN'].astype(str).str.strip()
df_giai_ngan['FORACID'] = df_giai_ngan['FORACID'].astype(str).str.strip()
pivot_full['CIF_KH_VAY'] = pivot_full['CIF_KH_VAY'].astype(str).str.strip()

# 2. Lọc các khế ước tại CRM32 khớp với FORACID
df_match = df_crm32_filtered[df_crm32_filtered['KHE_UOC'].isin(df_giai_ngan['FORACID'])].copy()

# 3. Lấy danh sách CIF của các khế ước khớp
ds_cif_tien_mat = df_match['CUSTSEQLN'].unique()

# 4. Đánh dấu vào pivot_full
pivot_full['GIẢI_NGÂN_TIEN_MAT'] = pivot_full['CIF_KH_VAY'].isin(ds_cif_tien_mat).map({True: 'x', False: ''})


In [ ]:
list_foracid = df_giai_ngan['FORACID'].astype(str).str.strip().unique()

In [ ]:
print(df_match)

In [ ]:
# # Chuẩn hóa giá trị KHE_UOC để đảm bảo không có khoảng trắng thừa
# list_khe_uoc_crm32 = df_crm32_filter['KHE_UOC'].astype(str).str.strip().unique().tolist()


In [ ]:
# pivot_full['GIẢI_NGÂN_TIỀN_MẶT'] = pivot_full['CIF_KH_VAY'].astype(str).str.strip().isin(list_khe_uoc_crm32).map({True: 'x', False: ''})


In [ ]:
# pivot_full['GIẢI_NGÂN_TIEN_MAT'] = pivot_full['CIF_KH_VAY'].astype(str).str.strip().isin(list_foracid).map({True: 'x', False: ''})


In [ ]:
# # Ép kiểu về string cho cả hai DataFrame
# df_cc_flag['CIF_KH_VAY'] = df_cc_flag['CIF_KH_VAY'].astype(str)
# pivot_full['CIF_KH_VAY'] = pivot_full['CIF_KH_VAY'].astype(str)

# # Ghép vào pivot_full
# pivot_full = pivot_full.merge(df_cc_flag, on='CIF_KH_VAY', how='left')
# pivot_full['Cầm cố tại TCTD khác'] = pivot_full['Cầm cố tại TCTD khác'].fillna('')


In [ ]:
df_cc_tctd = df_crm4_filtered[df_crm4_filtered['CAP_2'].str.contains('TCTD', case=False, na=False)]
df_cc_flag = df_cc_tctd[['CIF_KH_VAY']].drop_duplicates()
df_cc_flag['Cầm cố tại TCTD khác'] = 'x'

# Ghép vào pivot_full
pivot_full = pivot_full.merge(df_cc_flag, on='CIF_KH_VAY', how='left')
pivot_full['Cầm cố tại TCTD khác'] = pivot_full['Cầm cố tại TCTD khác'].fillna('')

In [ ]:
# Top 10 KHCN thay đổi ngày 12/05/2025
top5_khcn = pivot_full[pivot_full['CUSTTPCD'] == 'Ca nhan'].nlargest(10, 'DƯ NỢ')['CIF_KH_VAY']
pivot_full['Top 10 dư nợ KHCN'] = pivot_full['CIF_KH_VAY'].apply(lambda x: 'x' if x in top5_khcn.values else '')

# Top 10 KHDN thay đổi 12/05/2025
top5_khdn = pivot_full[pivot_full['CUSTTPCD'] == 'Doanh nghiep'].nlargest(10, 'DƯ NỢ')['CIF_KH_VAY']
pivot_full['Top 10 dư nợ KHDN'] = pivot_full['CIF_KH_VAY'].apply(lambda x: 'x' if x in top5_khdn.values else '')


In [ ]:
# import pandas as pd

# # Ngày đánh giá
# ngay_danh_gia = pd.to_datetime("2025-09-30")

# # Danh sách loại tài sản R34
# loai_ts_r34 = ['BĐS', 'MMTB', 'PTVT']

# # Chỉ áp dụng với các dòng thuộc R34
# mask_r34 = df_crm4_filtered['LOAI_TS'].isin(loai_ts_r34)

# # Chuẩn hóa ngày định giá
# df_crm4_filtered['VALUATION_DATE'] = pd.to_datetime(df_crm4_filtered['VALUATION_DATE'], errors='coerce')

# # Tính số ngày quá hạn: Ngày đánh giá - ngày định giá - 365 ngày
# df_crm4_filtered.loc[mask_r34, 'SO_NGAY_QUA_HAN'] = (
#     (ngay_danh_gia - df_crm4_filtered.loc[mask_r34, 'VALUATION_DATE']).dt.days - 365
# )

# # Tính số tháng quá hạn: (Ngày đánh giá - ngày định giá)/31 - 12
# df_crm4_filtered.loc[mask_r34, 'SO_THANG_QUA_HAN'] = (
#     ((ngay_danh_gia - df_crm4_filtered.loc[mask_r34, 'VALUATION_DATE']).dt.days / 31) - 12
# )

# # Lọc CIF có tài sản quá hạn > 30 ngày
# cif_quahan = df_crm4_filtered[
#     df_crm4_filtered['SO_NGAY_QUA_HAN'] > 30
# ]['CIF_KH_VAY'].unique()

# # Đánh dấu "x" trong pivot_full nếu có TSBĐ quá hạn định giá
# pivot_full['KH có TSBĐ quá hạn định giá'] = pivot_full['CIF_KH_VAY'].apply(
#     lambda x: 'X' if x in cif_quahan else ''
# )


ra quy định về định giá (11/8/20225)

In [ ]:
import pandas as pd

# Ngày đánh giá
ngay_danh_gia = pd.to_datetime("2025-09-30")

# Danh sách loại tài sản
loai_ts_r34 = ['BĐS', 'MMTB', 'PTVT']

# Chỉ áp dụng với các dòng thuộc R34
mask_r34 = df_crm4_filtered['LOAI_TS'].isin(loai_ts_r34)

# Chuẩn hóa ngày định giá
df_crm4_filtered['VALUATION_DATE'] = pd.to_datetime(df_crm4_filtered['VALUATION_DATE'], errors='coerce')

# Tính số ngày quá hạn: Ngày đánh giá - ngày định giá - 365 ngày
df_crm4_filtered.loc[mask_r34, 'SO_NGAY_QUA_HAN'] = (
    (ngay_danh_gia - df_crm4_filtered.loc[mask_r34, 'VALUATION_DATE']).dt.days - 365
)

# Tính số tháng quá hạn theo loại tài sản
df_crm4_filtered.loc[df_crm4_filtered['LOAI_TS'] == 'BĐS', 'SO_THANG_QUA_HAN'] = (
    ((ngay_danh_gia - df_crm4_filtered.loc[df_crm4_filtered['LOAI_TS'] == 'BĐS', 'VALUATION_DATE']).dt.days / 31) - 18
)

df_crm4_filtered.loc[df_crm4_filtered['LOAI_TS'].isin(['MMTB', 'PTVT']), 'SO_THANG_QUA_HAN'] = (
    ((ngay_danh_gia - df_crm4_filtered.loc[df_crm4_filtered['LOAI_TS'].isin(['MMTB', 'PTVT']), 'VALUATION_DATE']).dt.days / 31) - 12
)

# Lọc CIF có tài sản quá hạn > 30 ngày
cif_quahan = df_crm4_filtered[
    df_crm4_filtered['SO_NGAY_QUA_HAN'] > 30
]['CIF_KH_VAY'].unique()

# Đánh dấu "X" trong pivot_full nếu có TSBĐ quá hạn định giá
pivot_full['KH có TSBĐ quá hạn định giá'] = pivot_full['CIF_KH_VAY'].apply(
    lambda x: 'X' if x in cif_quahan else ''
)


muc 17

In [ ]:
df_sol = pd.read_excel('/content/vb/Muc17_Lop2_TSTC 4.xlsx')

In [ ]:
# Giả sử crm4_filter đã được xử lý trước đó
ds_secu = df_crm4_filtered['SECU_SRL_NUM'].dropna().unique()


In [ ]:
# Lọc df_17 sao cho mã trong cột C01 nằm trong danh sách đã lấy
df_17_filtered = df_sol[df_sol['C01'].isin(ds_secu)]


In [ ]:

# # Cho phép người dùng nhập tỉnh/thành cần so sánh
# dia_ban_kt = input("Nhập tên tỉnh/thành của đơn vị đang kiểm toán (ví dụ: Bạc Liêu): ").strip().lower()




28072025

In [ ]:
# Cho phép người dùng nhập nhiều tỉnh/thành
dia_ban_kt_input = input("Nhập tên tỉnh/thành của đơn vị đang kiểm toán (phân cách bằng dấu phẩy, ví dụ: Hồ Chí Minh, Long An): ")
# Tách chuỗi, chuẩn hóa chữ thường, bỏ khoảng trắng thừa
dia_ban_kt = [t.strip().lower() for t in dia_ban_kt_input.split(',') if t.strip()]

Nhập tên tỉnh/thành của đơn vị đang kiểm toán (phân cách bằng dấu phẩy, ví dụ: Hồ Chí Minh, Long An): thanh pho da nang


In [ ]:
# Bước 1: Lọc "Bất động sản" từ df_sol (Mục 17)
df_bds = df_17_filtered[df_17_filtered['C02'].str.strip() == 'Bat dong san'].copy()


In [ ]:
print(df_bds)

In [ ]:
# Bước 2: Đối chiếu mã tài sản giữa C01 (df_sol) và SECU_SRL_NUM (CRM4)
# Chỉ giữ tài sản thuộc Sol đó (có trong df_crm4)
df_bds_matched = df_bds[df_bds['C01'].isin(df_crm4['SECU_SRL_NUM'])].copy()


In [ ]:
print(df_bds_matched)

In [ ]:
# Bước 3: Tách tỉnh/thành phố từ cột C19 (lấy phần sau dấu phẩy cuối)
def extract_tinh_thanh(diachi):
    if pd.isna(diachi): return ''
    parts = diachi.split(',')
    return parts[-1].strip().lower() if parts else ''

df_bds_matched['TINH_TP_TSBD'] = df_bds_matched['C19'].apply(extract_tinh_thanh)



In [ ]:
df_bds_matched

28072025

In [ ]:
# Sửa lại điều kiện so sánh, dùng "not in"
# Cũng nên chuẩn hóa cột 'TINH_TP_TSBD' để đảm bảo so sánh chính xác
df_bds_matched['CANH_BAO_TS_KHAC_DIABAN'] = df_bds_matched['TINH_TP_TSBD'].apply(
    lambda x: 'x' if x and x.strip().lower() not in dia_ban_kt else ''
)

In [ ]:
# Bước 5: Lấy danh sách CIF có tài sản khác địa bàn (dựa vào mã tài sản map sang CRM4)
ma_ts_canh_bao = df_bds_matched[df_bds_matched['CANH_BAO_TS_KHAC_DIABAN'] == 'x']['C01'].unique()
cif_canh_bao = df_crm4[df_crm4['SECU_SRL_NUM'].isin(ma_ts_canh_bao)]['CIF_KH_VAY'].dropna().unique()



In [ ]:
# Bước 6: Đánh dấu trong pivot_full nếu khách có tài sản khác địa bàn (không dùng merge)
pivot_full['KH có TSBĐ khác địa bàn'] = pivot_full['CIF_KH_VAY'].apply(
    lambda x: 'x' if x in cif_canh_bao else ''
)

tieu chi 3


In [ ]:
df_55 = pd.read_excel('/content/vb/Muc55_1610.xlsx')
df_56 = pd.read_excel('/content/vb/Muc56_1610.xlsx')

In [ ]:
import pandas as pd

# -------------------- CHUẨN HÓA MỤC 55 - TẤT TOÁN --------------------
df_tt = df_55[['CUSTSEQLN', 'NMLOC', 'KHE_UOC', 'SOTIENGIAINGAN', 'NGAYGN', 'NGAYDH', 'NGAY_TT', 'LOAITIEN']].copy()
df_tt.columns = ['CIF', 'TEN_KHACH_HANG', 'KHE_UOC', 'SO_TIEN_GIAI_NGAN_VND',
                 'NGAY_GIAI_NGAN', 'NGAY_DAO_HAN', 'NGAY_TT', 'LOAI_TIEN_HD']
df_tt['GIAI_NGAN_TT'] = 'Tất toán'
df_tt['NGAY'] = pd.to_datetime(df_tt['NGAY_TT'], errors='coerce')

# -------------------- CHUẨN HÓA MỤC 56 - GIẢI NGÂN --------------------
df_gn = df_56[['CIF', 'TEN_KHACH_HANG', 'KHE_UOC', 'SO_TIEN_GIAI_NGAN_VND',
               'NGAY_GIAI_NGAN', 'NGAY_DAO_HAN', 'LOAI_TIEN_HD']].copy()
df_gn['GIAI_NGAN_TT'] = 'Giải ngân'
df_gn['NGAY_GIAI_NGAN'] = pd.to_datetime(df_gn['NGAY_GIAI_NGAN'], format='%Y%m%d', errors='coerce')
df_gn['NGAY_DAO_HAN'] = pd.to_datetime(df_gn['NGAY_DAO_HAN'], format='%Y%m%d', errors='coerce')
df_gn['NGAY'] = df_gn['NGAY_GIAI_NGAN']

# -------------------- GỘP 2 BẢNG LẠI --------------------
df_gop = pd.concat([df_tt, df_gn], ignore_index=True)
df_gop = df_gop[df_gop['NGAY'].notna()]  # Loại bỏ bản ghi không có ngày
df_gop = df_gop.sort_values(by=['CIF', 'NGAY', 'GIAI_NGAN_TT'])

# -------------------- ĐẾM SỐ KHẾ ƯỚC GN / TT THEO CIF & NGÀY --------------------
df_count = df_gop.groupby(['CIF', 'NGAY', 'GIAI_NGAN_TT']).size().unstack(fill_value=0).reset_index()
df_count['CO_CA_GN_VA_TT'] = ((df_count.get('Giải ngân', 0) > 0) & (df_count.get('Tất toán', 0) > 0)).astype(int)

# -------------------- DANH SÁCH CẢNH BÁO --------------------
ds_nhieu_tt_gn = df_count[
    (df_count.get('Giải ngân', 0) > 1) | (df_count.get('Tất toán', 0) > 1)
]['CIF'].unique()

ds_ca_gn_tt = df_count[df_count['CO_CA_GN_VA_TT'] == 1]['CIF'].unique()

# Ép kiểu cột CIF thành chuỗi để so sánh chính xác
df_count['CIF'] = df_count['CIF'].astype(str)
df_gop['CIF'] = df_gop['CIF'].astype(str)
df_tt['CIF'] = df_tt['CIF'].astype(str)
df_gn['CIF'] = df_gn['CIF'].astype(str)

# Tạo danh sách cảnh báo đã ép kiểu
ds_nhieu_tt_gn = df_count[
    (df_count.get('Giải ngân', 0) > 1) | (df_count.get('Tất toán', 0) > 1)
]['CIF'].astype(str).unique()

ds_ca_gn_tt = df_count[df_count['CO_CA_GN_VA_TT'] == 1]['CIF'].astype(str).unique()

# Ép kiểu CIF_KH_VAY trong pivot để so sánh
pivot_full['CIF_KH_VAY'] = pivot_full['CIF_KH_VAY'].astype(str)

# Đánh dấu vào pivot
pivot_full['KH có cả GNG và TT trong 1 ngày'] = pivot_full['CIF_KH_VAY'].apply(
    lambda x: 'x' if x in ds_ca_gn_tt else ''
)


/tmp/ipython-input-3935960843.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tt['NGAY'] = pd.to_datetime(df_tt['NGAY_TT'], errors='coerce')


In [ ]:
import pandas as pd
import numpy as np

# 1. Đọc dữ liệu chậm trả
df_delay = pd.read_excel("/content/vb/Muc57_1610.xlsx")

# 2. Chuẩn hóa ngày
df_delay['NGAY_DEN_HAN_TT'] = pd.to_datetime(df_delay['NGAY_DEN_HAN_TT'], errors='coerce')
df_delay['NGAY_THANH_TOAN'] = pd.to_datetime(df_delay['NGAY_THANH_TOAN'], errors='coerce')

# 3. Điền ngày thanh toán = ngày đánh giá nếu thiếu
ngay_danh_gia = pd.to_datetime("2025-09-30")
df_delay['NGAY_THANH_TOAN_FILL'] = df_delay['NGAY_THANH_TOAN'].fillna(ngay_danh_gia)

# 4. Tính số ngày chậm trả
df_delay['SO_NGAY_CHAM_TRA'] = (df_delay['NGAY_THANH_TOAN_FILL'] - df_delay['NGAY_DEN_HAN_TT']).dt.days

# 5. Lọc giai đoạn 2023–2025
mask_period = df_delay['NGAY_DEN_HAN_TT'].dt.year.between(2023, 2025)
df_delay = df_delay[mask_period].copy()

# 6. Gộp dư nợ & nhóm nợ từ pivot_full
df_crm32 = pivot_full.copy()
df_crm32 = df_crm32.rename(columns={'CIF_KH_VAY': 'CIF_ID'})

# --- Sửa lỗi kiểu dữ liệu trước khi merge lần đầu ---
df_crm32['CIF_ID'] = df_crm32['CIF_ID'].astype(str)
df_delay['CIF_ID'] = df_delay['CIF_ID'].astype(str)

df_delay = df_delay.merge(
    df_crm32[['CIF_ID', 'DƯ NỢ', 'NHOM_NO']],
    on='CIF_ID', how='left'
)

# 7. Chỉ lấy nợ nhóm 1
df_delay = df_delay[df_delay['NHOM_NO'] == 1].copy()

# 8. Gán cấp độ chậm trả
def cap_cham_tra(days):
    if pd.isna(days):
        return None
    elif days >= 10:
        return '>=10'
    elif days >= 4:
        return '4-9'
    elif days > 0:
        return '<4'
    else:
        return None

df_delay['CAP_CHAM_TRA'] = df_delay['SO_NGAY_CHAM_TRA'].apply(cap_cham_tra)

# Loại bỏ các dòng có CAP_CHAM_TRA là None
df_delay = df_delay.dropna(subset=['CAP_CHAM_TRA']).copy()


# 9. Ưu tiên mỗi CIF_ID + ngày chỉ 1 cấp độ cao nhất
df_delay['NGAY'] = df_delay['NGAY_DEN_HAN_TT'].dt.date
df_delay.sort_values(['CIF_ID', 'NGAY', 'CAP_CHAM_TRA'],
                     key=lambda s: s.map({'>=10':0, '4-9':1, '<4':2}),
                     inplace=True)
df_unique = df_delay.drop_duplicates(subset=['CIF_ID', 'NGAY'], keep='first').copy()

# 10. Đếm số lần theo cấp độ
df_dem = df_unique.groupby(['CIF_ID', 'CAP_CHAM_TRA']).size().unstack(fill_value=0)

# 11. Gắn cờ vào 2 cột
# Sử dụng .get() với giá trị mặc định 0 để tránh KeyError nếu cột không tồn tại sau unstack
df_dem['KH Phát sinh chậm trả > 10 ngày'] = np.where(df_dem.get('>=10', 0) > 0, 'x', '')
df_dem['KH Phát sinh chậm trả 4-9 ngày'] = np.where(
    (df_dem.get('>=10', 0) == 0) & (df_dem.get('4-9', 0) > 0), 'x', ''
)

# 12. Gộp vào pivot_full
# --- Sửa lỗi kiểu dữ liệu trước khi merge lần cuối ---
pivot_full['CIF_KH_VAY'] = pivot_full['CIF_KH_VAY'].astype(str)

# Gộp các cột cờ từ df_dem vào pivot_full
cols_to_merge = ['KH Phát sinh chậm trả > 10 ngày', 'KH Phát sinh chậm trả 4-9 ngày']

# Chỉ chọn các cột thực sự có trong df_dem (để an toàn, mặc dù .get() ở bước 11 đã giúp)
cols_to_merge_existing = [col for col in cols_to_merge if col in df_dem.columns]

if cols_to_merge_existing: # Chỉ merge nếu có cột để merge
    pivot_full = pivot_full.merge(
        df_dem[cols_to_merge_existing],
        left_on='CIF_KH_VAY', right_index=True, how='left'
    )

# 13. Điền trống nếu không có
# Điền giá trị rỗng ('') vào các ô NaN trong các cột cờ đã được gộp
for col in cols_to_merge_existing:
     pivot_full[col] = pivot_full[col].fillna('')

# Bạn có thể thêm print(pivot_full.head()) ở cuối để xem kết quả


In [ ]:
with pd.ExcelWriter('/content/KQ_1610_.xlsx', engine='openpyxl') as writer:
    df_crm4_filtered.to_excel(writer, sheet_name='df_crm4_LOAI_TS', index=False)
    pivot_final.to_excel(writer, sheet_name='KQ_CRM4', index=False)
    pivot_merge.to_excel(writer, sheet_name='Pivot_crm4', index=False)
    df_crm32_filtered.to_excel(writer, sheet_name='df_crm32_LOAI_TS', index=False)
    pivot_full.to_excel(writer, sheet_name='KQ_KH', index=False)
    pivot_mucdich.to_excel(writer, sheet_name='Pivot_crm32', index=False)

    #df_r34.to_excel(writer, sheet_name='tieu chi 1_dot3', index=False)
    df_delay.to_excel(writer, sheet_name='tieu chi 4', index=False)
    df_gop.to_excel(writer, sheet_name='tieu chi 3_dot3', index=False)
    df_count.to_excel(writer, sheet_name='tieu chi 3_dot3_1', index=False)
    df_bds_matched.to_excel(writer, sheet_name='tieu chi 2_dot3', index=False)